# HISTORY

#### Context

* The Laser Interferometer Gravitational-Wave Observatory (LIGO) was designed to open the field of gravitational-wave astrophysics through the direct detection of gravitational waves predicted by Einstein’s General Theory of Relativity. LIGO’s multi-kilometer-scale gravitational wave detectors use laser interferometry to measure the minute ripples in space-time caused by passing gravitational waves from cataclysmic cosmic events such as colliding neutron stars or black holes, or by supernovae. LIGO consists of two widely-separated interferometers within the United States—one in Hanford, Washington and the other in Livingston, Louisiana—operated in unison to detect gravitational waves.

* This dataset is made up of LIGO images that have been classified by people as part of the Gravity Spy Zooniverse project and they've been placed into train,validation and test folders,ready for machine learning.

# PACKAGES AND LIBRARIES

In [ ]:
#GENERAL
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
#PATH PROCESS
import os
import os.path
from pathlib import Path
import glob
#IMAGE PROCESS
from PIL import Image
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
from keras.applications.vgg16 import preprocess_input, decode_predictions
#SCALER & TRANSFORMATION
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from keras import regularizers
from sklearn.preprocessing import LabelEncoder
#ACCURACY CONTROL
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
#OPTIMIZER
from keras.optimizers import RMSprop,Adam,Optimizer,Optimizer, SGD
#MODEL LAYERS
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization,MaxPooling2D,BatchNormalization,\
                        Permute, TimeDistributed, Bidirectional,GRU, SimpleRNN,\
LSTM, GlobalAveragePooling2D, SeparableConv2D, ZeroPadding2D, Convolution2D, ZeroPadding2D,Reshape, Conv2DTranspose, LeakyReLU, GaussianNoise
from keras import models
from keras import layers
import tensorflow as tf
from keras.applications import VGG16,VGG19,inception_v3
from keras import backend as K
from keras.utils import plot_model
from keras.datasets import mnist
import keras
#SKLEARN CLASSIFIER
from xgboost import XGBClassifier, XGBRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.cross_decomposition import PLSRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV
#IGNORING WARNINGS
from warnings import filterwarnings
filterwarnings("ignore",category=DeprecationWarning)
filterwarnings("ignore", category=FutureWarning) 
filterwarnings("ignore", category=UserWarning)

# PATH & LABEL PROCESS

#### MAIN PATH

In [ ]:
Wave_Train_Data = Path("../input/gravity-spy-gravitational-waves/train/train")
Wave_Test_Data = Path("../input/gravity-spy-gravitational-waves/test/test")
Wave_Validation_Data = Path("../input/gravity-spy-gravitational-waves/validation/validation")

#### PNG PATH

In [ ]:
Train_PNG_Path = list(Wave_Train_Data.glob(r"*/*.png"))
Test_PNG_Path = list(Wave_Test_Data.glob(r"*/*.png"))
Validation_PNG_Path = list(Wave_Validation_Data.glob(r"*/*.png"))

#### LABELS

In [ ]:
Train_PNG_Labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1],Train_PNG_Path))
Test_PNG_Labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1],Test_PNG_Path))
Validation_PNG_Labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1],Validation_PNG_Path))

#### TO SERIES

##### PATH

In [ ]:
Train_PNG_Path_Series = pd.Series(Train_PNG_Path,name="PNG").astype(str)
Test_PNG_Path_Series = pd.Series(Test_PNG_Path,name="PNG").astype(str)
Validation_PNG_Path_Series = pd.Series(Validation_PNG_Path,name="PNG").astype(str)

##### LABELS

In [ ]:
Train_PNG_Labels_Series = pd.Series(Train_PNG_Labels,name="CATEGORY")
Test_PNG_Labels_Series = pd.Series(Test_PNG_Labels,name="CATEGORY")
Validation_PNG_Labels_Series = pd.Series(Validation_PNG_Labels,name="CATEGORY")

In [ ]:
print(Train_PNG_Labels_Series.value_counts())
print("---"*20)
print(Train_PNG_Labels_Series.value_counts().sum())

In [ ]:
print(Test_PNG_Labels_Series.value_counts())
print("---"*20)
print(Test_PNG_Labels_Series.value_counts().sum())

In [ ]:
print(Validation_PNG_Labels_Series.value_counts())
print("---"*20)
print(Validation_PNG_Labels_Series.value_counts().sum())

#### TO DATAFRAME

In [ ]:
Main_Train_Data = pd.concat([Train_PNG_Path_Series,Train_PNG_Labels_Series],axis=1)
Main_Test_Data = pd.concat([Test_PNG_Path_Series,Test_PNG_Labels_Series],axis=1)
Main_Validation_Data = pd.concat([Validation_PNG_Path_Series,Validation_PNG_Labels_Series],axis=1)

In [ ]:
print(Main_Train_Data.head(-1))

In [ ]:
print(Main_Test_Data.head(-1))

In [ ]:
print(Main_Validation_Data.head(-1))

#### SHUFFLING

In [ ]:
Main_Train_Data = Main_Train_Data.sample(frac=1).reset_index(drop=True)

In [ ]:
print(Main_Train_Data.head(-1))

In [ ]:
Main_Test_Data = Main_Test_Data.sample(frac=1).reset_index(drop=True)

In [ ]:
print(Main_Test_Data.head(-1))

In [ ]:
Main_Validation_Data = Main_Validation_Data.sample(frac=1).reset_index(drop=True)

In [ ]:
print(Main_Validation_Data.head(-1))

# VISION

In [ ]:
plt.style.use("classic")

In [ ]:
example_IMG = cv2.imread(Main_Train_Data["PNG"][22033])
plt.xlabel(example_IMG.shape)
plt.ylabel(example_IMG.size)
plt.title(Main_Train_Data["CATEGORY"][22033])
plt.imshow(example_IMG)

In [ ]:
example_IMG = cv2.imread(Main_Train_Data["PNG"][23])
plt.xlabel(example_IMG.shape)
plt.ylabel(example_IMG.size)
plt.title(Main_Train_Data["CATEGORY"][23])
plt.imshow(example_IMG)

In [ ]:
example_IMG = cv2.imread(Main_Train_Data["PNG"][1009])
plt.xlabel(example_IMG.shape)
plt.ylabel(example_IMG.size)
plt.title(Main_Train_Data["CATEGORY"][1009])
plt.imshow(example_IMG)

In [ ]:
example_IMG = cv2.imread(Main_Train_Data["PNG"][101])
plt.xlabel(example_IMG.shape)
plt.ylabel(example_IMG.size)
plt.title(Main_Train_Data["CATEGORY"][101])
plt.imshow(example_IMG)

In [ ]:
figure,axis = plt.subplots(nrows=5,ncols=5,figsize=(13,13))

for i,ax in enumerate(axis.flat):
    IMG = plt.imread(Main_Train_Data["PNG"][i])
    ax.imshow(IMG)
    ax.set_xlabel(IMG.shape)
    ax.set_ylabel(IMG.size)
    ax.set_title(Main_Train_Data["CATEGORY"][i])
plt.tight_layout()
plt.show()

In [ ]:
figure,axis = plt.subplots(nrows=5,ncols=5,figsize=(13,13))

for i,ax in enumerate(axis.flat):
    IMG = cv2.imread(Main_Train_Data["PNG"][i])
    ax.imshow(IMG)
    ax.set_xlabel(round(np.mean(IMG)))
    ax.set_ylabel(IMG.shape)
    ax.set_title(Main_Train_Data["CATEGORY"][i])
plt.tight_layout()
plt.show()

In [ ]:
figure,axis = plt.subplots(nrows=5,ncols=5,figsize=(13,13))

for i,ax in enumerate(axis.flat):
    IMG = cv2.imread(Main_Train_Data["PNG"][i])
    Canny_IMG = cv2.Canny(IMG,10,100)
    ax.imshow(Canny_IMG)
    ax.set_xlabel(Canny_IMG.shape)
    ax.set_ylabel(Canny_IMG.size)
    ax.set_title(Main_Train_Data["CATEGORY"][i])
plt.tight_layout()
plt.show()

In [ ]:
figure,axis = plt.subplots(nrows=5,ncols=5,figsize=(13,13))

for i,ax in enumerate(axis.flat):
    IMG = cv2.imread(Main_Train_Data["PNG"][i])
    _,Threshold_IMG = cv2.threshold(IMG,90,255,cv2.THRESH_BINARY)
    ax.imshow(Threshold_IMG)
    ax.set_xlabel(round(np.mean(Threshold_IMG)))
    ax.set_ylabel(Threshold_IMG.shape)
    ax.set_title(Main_Train_Data["CATEGORY"][i])
plt.tight_layout()
plt.show()

In [ ]:
figure,axis = plt.subplots(nrows=5,ncols=5,figsize=(13,13))

for i,ax in enumerate(axis.flat):
    IMG = cv2.imread(Main_Train_Data["PNG"][i],0)
    AdaptiveThreshold_IMG = cv2.adaptiveThreshold(IMG,200,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
    ax.imshow(AdaptiveThreshold_IMG)
    ax.set_xlabel(round(np.mean(AdaptiveThreshold_IMG)))
    ax.set_ylabel(AdaptiveThreshold_IMG.shape)
    ax.set_title(Main_Train_Data["CATEGORY"][i])
plt.tight_layout()
plt.show()

In [ ]:
Background_SUB = cv2.createBackgroundSubtractorMOG2()

figure,axis = plt.subplots(nrows=5,ncols=5,figsize=(13,13))

for i,ax in enumerate(axis.flat):
    IMG = cv2.imread(Main_Train_Data["PNG"][i])
    SUB_IMG = Background_SUB.apply(IMG)
    ax.imshow(SUB_IMG)
    ax.set_xlabel(round(np.mean(SUB_IMG)))
    ax.set_ylabel(SUB_IMG.shape)
    ax.set_title(Main_Train_Data["CATEGORY"][i])
plt.tight_layout()
plt.show()

In [ ]:
figure,axis = plt.subplots(nrows=5,ncols=5,figsize=(13,13))

for i,ax in enumerate(axis.flat):
    IMG = cv2.imread(Main_Train_Data["PNG"][i])
    Canny_IMG = cv2.Canny(IMG,10,100)
    contour,_ = cv2.findContours(Canny_IMG,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    draw_contour = cv2.drawContours(Canny_IMG,contour,-1,(255,0,0),2)
    ax.imshow(draw_contour)
    ax.set_xlabel(round(np.mean(draw_contour)))
    ax.set_ylabel(draw_contour.shape)
    ax.set_title(Main_Train_Data["CATEGORY"][i])
plt.tight_layout()
plt.show()

In [ ]:
figure,axis = plt.subplots(nrows=5,ncols=5,figsize=(13,13))

for i,ax in enumerate(axis.flat):
    IMG = cv2.imread(Main_Train_Data["PNG"][i])
    IMG = cv2.resize(IMG,(28,28))
    _,threshold_IMG = cv2.threshold(IMG,90,255,cv2.THRESH_BINARY_INV)
    threshold_IMG = cv2.resize(threshold_IMG,(28,28))
    mask = cv2.inRange(IMG,IMG,threshold_IMG)
    Sub_Mask = cv2.bitwise_and(IMG,IMG,mask=mask)
    ax.imshow(Sub_Mask)
    ax.set_xlabel(round(np.mean(Sub_Mask)))
    ax.set_ylabel(Sub_Mask.shape)
    ax.set_title(Main_Train_Data["CATEGORY"][i])
plt.tight_layout()
plt.show()

In [ ]:
figure,axis = plt.subplots(nrows=5,ncols=5,figsize=(13,13))

for i,ax in enumerate(axis.flat):
    IMG = cv2.imread(Main_Train_Data["PNG"][i])
    IMG = cv2.resize(IMG,(28,28))
    _,threshold_IMG = cv2.threshold(IMG,90,255,cv2.THRESH_BINARY_INV)
    threshold_IMG = cv2.resize(threshold_IMG,(28,28))
    mask = cv2.inRange(IMG,IMG,threshold_IMG)
    Sub_Mask = cv2.bitwise_and(IMG,IMG,mask=mask)
    Canny_IMG = cv2.Canny(Sub_Mask,10,100)
    contour,_ = cv2.findContours(Canny_IMG,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    draw_contour = cv2.drawContours(Canny_IMG,contour,-1,(255,0,0),2)
    ax.imshow(draw_contour)
    ax.set_xlabel(round(np.mean(draw_contour)))
    ax.set_ylabel(draw_contour.shape)
    ax.set_title(Main_Train_Data["CATEGORY"][i])
plt.tight_layout()
plt.show()

In [ ]:
figure,axis = plt.subplots(nrows=5,ncols=5,figsize=(13,13))

for i,ax in enumerate(axis.flat):
    IMG = cv2.imread(Main_Train_Data["PNG"][i])
    IMG = cv2.resize(IMG,(28,28))
    _,threshold_IMG = cv2.threshold(IMG,90,255,cv2.THRESH_BINARY_INV)
    threshold_IMG = cv2.resize(threshold_IMG,(28,28))
    mask = cv2.inRange(IMG,IMG,threshold_IMG)
    Sub_Mask = cv2.bitwise_and(IMG,IMG,mask=mask)
    Canny_IMG = cv2.Canny(Sub_Mask,10,100)
    contour,_ = cv2.findContours(Canny_IMG,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    
    for cnt in contour:
        x,y,w,h = cv2.boundingRect(cnt)
        cv2.rectangle(Sub_Mask,(x,y),(x+w,y+h),(255,0,0),1)
        
    ax.imshow(Sub_Mask)
    ax.set_xlabel(round(np.mean(Sub_Mask)))
    ax.set_ylabel(Sub_Mask.shape)
    ax.set_title(Main_Train_Data["CATEGORY"][i])
plt.tight_layout()
plt.show()

In [ ]:
figure,axis = plt.subplots(nrows=5,ncols=5,figsize=(13,13))

for i,ax in enumerate(axis.flat):
    IMG = cv2.imread(Main_Train_Data["PNG"][i])
    IMG = cv2.resize(IMG,(180,180))
    _,threshold_IMG = cv2.threshold(IMG,90,255,cv2.THRESH_BINARY_INV)
    threshold_IMG = cv2.resize(threshold_IMG,(180,180))
    mask = cv2.inRange(IMG,IMG,threshold_IMG)
    Sub_Mask = cv2.bitwise_and(IMG,IMG,mask=mask)
    Canny_IMG = cv2.Canny(Sub_Mask,10,100)
    contour,_ = cv2.findContours(Canny_IMG,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    
    for cnt in contour:
        x,y,w,h = cv2.boundingRect(cnt)
        cv2.rectangle(IMG,(x,y),(x+w,y+h),(255,0,0),1)
        
    ax.imshow(IMG)
    ax.set_xlabel(round(np.mean(IMG)))
    ax.set_ylabel(IMG.shape)
    ax.set_title(Main_Train_Data["CATEGORY"][i])
plt.tight_layout()
plt.show()

# DATA PROCESS

#### GENERAL SPLITTING

In [ ]:
Y_Validation_Data = Train_PNG_Path[5000:10000]

In [ ]:
Y_Validation_Data_Series = pd.Series(Y_Validation_Data,name="PNG").astype(str)

In [ ]:
print(Y_Validation_Data_Series.shape)

In [ ]:
X_Training_Data = Train_PNG_Path[:5000]

In [ ]:
X_Training_Data_Series = pd.Series(X_Training_Data,name="PNG").astype(str)

In [ ]:
print(X_Training_Data_Series.shape)

#### TRANSFORMATION

In [ ]:
Transformated_X = []

for IMG_X in X_Training_Data_Series:
    X_IMG = cv2.imread(IMG_X)
    X_IMG = X_IMG / 255.
    R_IMG = cv2.resize(X_IMG,(28,28))
    Transformated_X.append(R_IMG)

In [ ]:
Transformated_Y = []

for IMG_Y in Y_Validation_Data_Series:
    Y_IMG = cv2.imread(IMG_Y)
    Y_IMG = Y_IMG / 255.
    RY_IMG = cv2.resize(Y_IMG,(28,28))
    Transformated_Y.append(RY_IMG)

In [ ]:
figure = plt.figure(figsize=(5,5))
plt.xlabel(Transformated_X[2].shape)
plt.ylabel(Transformated_X[2].size)
plt.imshow(Transformated_X[2])

In [ ]:
figure = plt.figure(figsize=(5,5))
plt.xlabel(Transformated_Y[2].shape)
plt.ylabel(Transformated_Y[2].size)
plt.imshow(Transformated_Y[2])

#### MAIN DATA

In [ ]:
X_Train = np.array(Transformated_X)
Y_Train = np.array(Transformated_Y)

In [ ]:
print(X_Train.shape)

In [ ]:
print(Y_Train.shape)

# AUTO ENCODER PROCESS

#### STRUCTURE

In [ ]:
encoder = Sequential()
encoder.add(Flatten(input_shape=[28,28,3]))
encoder.add(Dense(400,activation="relu"))
encoder.add(Dense(300,activation="relu"))
encoder.add(Dense(200,activation="relu"))
encoder.add(Dense(100,activation="relu"))
encoder.add(Dense(75,activation="relu"))
encoder.add(Dense(50,activation="relu"))
encoder.add(Dense(25,activation="relu"))

In [ ]:
print(encoder.summary())

In [ ]:
decoder = Sequential()
decoder.add(Dense(50,input_shape=[25],activation="relu"))
decoder.add(Dense(75,activation="relu"))
decoder.add(Dense(100,activation="relu"))
decoder.add(Dense(200,activation="relu"))
decoder.add(Dense(300,activation="relu"))
decoder.add(Dense(400,activation="relu"))
decoder.add(Dense(2352,activation="sigmoid"))
decoder.add(Reshape([28,28,3]))

In [ ]:
print(decoder.summary())

In [ ]:
Auto_Encoder = Sequential([encoder,decoder])

In [ ]:
print(Auto_Encoder.summary())

In [ ]:
Auto_Encoder.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [ ]:
Auto_Encoder.fit(X_Train,X_Train,validation_data=(Y_Train,Y_Train),epochs=10)

#### PREDICTION

In [ ]:
Predict_IMG = Auto_Encoder.predict(Y_Train[:10])

In [ ]:
print(Y_Train[:10].shape)

In [ ]:
print(Predict_IMG[1].shape)

In [ ]:
prediction_img = 5
print("NORMAL")
plt.imshow(Y_Train[prediction_img])
plt.show()
print("Auto Encoder")
plt.imshow(Predict_IMG[prediction_img])

In [ ]:
figure,axis = plt.subplots(nrows=2,ncols=5,figsize=(10,10))

for P_Image,ax in enumerate(axis.flat):
    ax.imshow(Predict_IMG[P_Image])

#### NON SEEN PREDICT

In [ ]:
Non_Seen_IMG = Main_Train_Data["PNG"][20000]
IMG_T = cv2.imread(Non_Seen_IMG)
IMG_T = IMG_T / 255.
IMG_T = cv2.resize(IMG_T,(28,28))

In [ ]:
IMG_T = np.array(IMG_T)

In [ ]:
IMG_T = IMG_T.reshape(-1,28,28,3)

In [ ]:
print(IMG_T.shape)

In [ ]:
Pre_Non_Seen = Auto_Encoder.predict(IMG_T)

In [ ]:
print("NORMAL")
plt.imshow(IMG_T[0])
plt.show()
print("Auto Encoder")
plt.imshow(Pre_Non_Seen[0])

# DC-GAN PROCESS

In [ ]:
Generator_Input = keras.Input(shape=(28,))
x = layers.Dense(128*14*14)(Generator_Input)
x = layers.LeakyReLU()(x)
x = layers.Reshape((14,14,128))(x)

x = layers.Conv2D(256,5,padding="same")(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2DTranspose(256,4,padding="same",strides=2)(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(256,5,padding="same")(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256,4,padding="same")(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256,3,padding="same")(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(3,7,padding="same",activation="tanh")(x)

Generator = keras.models.Model(Generator_Input,x)

In [ ]:
Discriminator_Input = layers.Input(shape=(28,28,3))
x = layers.Conv2D(128,3)(Discriminator_Input)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(128,4)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128,4)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128,3)(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(128,3)(x)
x = layers.LeakyReLU()(x)
x = layers.Flatten()(x)

x = layers.Dense(1,activation="sigmoid")(x)

Discriminator = keras.models.Model(Discriminator_Input,x)

In [ ]:
Discriminator.compile(optimizer=RMSprop(lr=0.0004,clipvalue=1.0,decay=1e-8),loss="binary_crossentropy")

In [ ]:
Discriminator.trainable = False

Gan_Input = keras.Input(shape=(28,))
Gan_Output = Discriminator(Generator(Gan_Input))
GAN_Model = keras.models.Model(Gan_Input,Gan_Output)

In [ ]:
GAN_Model.compile(optimizer=RMSprop(lr=0.0004,clipvalue=1.0,decay=1e-8),loss="binary_crossentropy")

In [ ]:
os.mkdir("wave8_new")

In [ ]:
start = 0
batch_size = 32
iterations = 10000
dim_size = 28

for step in range(iterations):
    random_noise_vector = np.random.normal(size=(batch_size,dim_size)) #32
    generation_images = Generator.predict(random_noise_vector)
    
    stop = start + batch_size
    real_images = X_Train[start:stop]
    
    combined_images = np.concatenate([generation_images,real_images])
    
    labels = np.concatenate([np.ones((batch_size,1)),np.zeros((batch_size,1))])
    labels += 0.05 * np.random.random(labels.shape)
    
    D_loss = Discriminator.train_on_batch(combined_images,labels)
    
    random_noise_vector = np.random.normal(size=(batch_size,dim_size)) #32
    
    misleading_targets = np.zeros((batch_size,1))
    
    a_loss = GAN_Model.train_on_batch(random_noise_vector,misleading_targets)
    
    start += batch_size
    
    if start > len(X_Train) - batch_size:
        start = 0
    
    if step % 100 == 0:
        GAN_Model.save_weights("GAN_ONE.h5")
        
        print("DISC_LOSS: ", D_loss)
        print("ADVERSARIAL_LOSS: ", a_loss)
        
        Img = image.array_to_img(generation_images[0] * 255.,scale=False)
        Img.save(os.path.join("./wave8_new","FAKE"+str(step)+".png"))
        
        Img = image.array_to_img(real_images[0] * 255.,scale=False)
        Img.save(os.path.join("./wave8_new","REAL"+str(step)+".png"))

In [ ]:
Exp_output = Path("./wave8_new")

In [ ]:
list_output = list(Exp_output.glob(r"*.png"))

In [ ]:
list_output_series = pd.Series(list_output,name="PNG").astype(str)

In [ ]:
noise_PR = tf.random.normal(shape=[10,28])

In [ ]:
print(noise_PR.shape)

In [ ]:
plt.imshow(noise_PR)

In [ ]:
PR_Images = Generator(noise_PR)

In [ ]:
figure, axes = plt.subplots(nrows=2,ncols=5,figsize=(10,10))

for i,ax in enumerate(axes.flat):
    ax.imshow(PR_Images[i],cmap="Greys_r")
    ax.set_xlabel(PR_Images[i].shape)
plt.tight_layout()
plt.show()

In [ ]:
figure = plt.figure(figsize=(10,10))
plt.imshow(PR_Images[7])

In [ ]:
figure, axes = plt.subplots(nrows=4,ncols=4,figsize=(10,10))

for i,ax in enumerate(axes.flat):
    List_Gen_Image = cv2.imread(list_output_series[i])
    ax.imshow(List_Gen_Image,cmap="Greys_r")
    ax.set_xlabel(List_Gen_Image.shape)
    ax.set_ylabel(List_Gen_Image.size)
plt.tight_layout()
plt.show()